In [2]:
import  pandas  as pd
import  numpy  as np
import statsmodels.formula.api as smf
from prettytable import PrettyTable

In [3]:
df = pd.read_csv("resp1.csv",delimiter='@')
df['response']=np.int64((df['responseCategory']=='CR')|(df['responseCategory']=='PR'))
df['TRTPN'].replace([2],[0], inplace=True)         #Treatment1 = 1  / Treatent2 = 0
df['gender'] = np.int64(df['gender'] == 'MALE')    # Male = 1   / Female = 0 
                                
df.head()

,gender,SITEID,SUBJID,TRTPN,responseCategory,response
0,1,1,27,0,SD,0
1,0,1,39,1,PD,0
2,1,1,126,0,PD,0
3,1,1,154,1,SD,0
4,0,1,161,1,PD,0


In [4]:
model1 = smf.logit("response ~ TRTPN ", data = df).fit()

model1.summary()

Optimization terminated successfully.
         Current function value: 0.432137
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               response   No. Observations:                  582
Model:                          Logit   Df Residuals:                      580
Method:                           MLE   Df Model:                            1
Date:                Fri, 22 Oct 2021   Pseudo R-squ.:                0.009915
Time:                        10:27:40   Log-Likelihood:                -251.50
converged:                       True   LL-Null:                       -254.02
Covariance Type:            nonrobust   LLR p-value:                   0.02481
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.9538      0.178    -10.971      0.000      -2.303      -1.605
TRTPN          0.5153      0.232      2.222      0.026       0.061       0.970
==============================================================================
"""

In [5]:
model1_odds = pd.DataFrame(np.exp(model1.params), columns= ['OR'])
model1_odds['z-value']= model1.pvalues
model1_odds[['2.5%', '97.5%']] = np.exp(model1.conf_int())

model1_odds

,OR,z-value,2.5%,97.5%
Intercept,0.141732,5.259220e-28,0.099973,0.200935
TRTPN,1.674200,2.631187e-02,1.062565,2.637903


## Definition of Odds Ratio:
An odds ratio (OR) is a measure of association between an exposure and an outcome. The OR represents the odds that an outcome will occur given a particular exposure, compared to the odds of the outcome occurring in the absence of that exposure. In other words, the OR is a measure of the odds of an event happening in one group compared to the odds of the same event happening in another group.

## Definition of confidence interval:
The confidence interval (CI) is a range of values that’s likely to include a population value with a certain degree of confidence. The 95% confidence interval (CI) is used to estimate the precision of the OR. A large CI indicates a low level of precision of the OR, whereas a small CI indicates a higher precision of the OR.

## How it is related with regression coefficients estimated in this model?
When a logistic regression is calculated, the regression coefficient (b1) is the estimated increase in the log odds of the outcome per unit increase in the value of the exposure. In other words, the exponential function of the regression coefficient (e^b1) is the odds ratio associated with a one-unit increase in the exposure.

OR=1.674200 - 
  It tell us, how much treatment 1 over treatment 2 increases the odds of a subject having response to treatment.

In [6]:
model2 = smf.logit("response ~ TRTPN + gender + TRTPN*gender ", data = df).fit()

model2.summary()


Optimization terminated successfully.
         Current function value: 0.431351
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               response   No. Observations:                  582
Model:                          Logit   Df Residuals:                      578
Method:                           MLE   Df Model:                            3
Date:                Fri, 22 Oct 2021   Pseudo R-squ.:                 0.01172
Time:                        10:27:47   Log-Likelihood:                -251.05
converged:                       True   LL-Null:                       -254.02
Covariance Type:            nonrobust   LLR p-value:                    0.1139
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.8909      0.268     -7.050      0.000      -2.417      -1.365
TRTPN            0.3067      0.349      0.878      0.380      -0.378       0.992
gender          -0.1106      0.359     -0.308      0.758      -0.814       0.592
TRTPN:gender     0.3814      0.468      0.816      0.415      -0.535       1.298
================================================================================
"""

r=b0 + b1*T + b2*G + b3*T*G

##  Conditional odds ratio
C1 : treatment 1 vs treatment 2 | Male      / b1+b3

C2 : treatment 1 vs treatment 2 | Female    / b1

C3 : Female vs Male | treatment 1           / b2+b3

C4 : Female vs Male | treatment 2           / b2

In [7]:
b1b3=model2.params[1]+model2.params[3]
b2b3=model2.params[2]+model2.params[3]

c0=[np.exp(model1.params[1]), np.exp(model1.conf_int())[0][1], np.exp(model1.conf_int())[1][1]]
c1=[np.exp(b1b3), np.exp(b1b3-1.96*model2.bse[3]), np.exp(b1b3+1.96*model2.bse[3])]
c2=[np.exp(model2.params[1]), np.exp(model2.conf_int())[0][1], np.exp(model2.conf_int())[1][1]]
c3=[np.exp(b2b3), np.exp(b2b3-1.96*model2.bse[3]), np.exp(b2b3+1.96*model2.bse[3]) ]
c4=[np.exp(model2.params[2]), np.exp(model2.conf_int())[0][2], np.exp(model2.conf_int())[1][2]]
pval=model2.pvalues[3]

In [8]:
#Report all results
y = PrettyTable()

y = PrettyTable(["RESPONSE RATE", ""] )
y.title = 'Logistic regression for subjects response'

y.add_row(["ODDS RATIO (95% CI): Treatment 1 VS. Treatment 2\n","%.2f (%.2f %.2f)"%(c0[0],c0[1],c0[2])])

y.add_row(["ODDS RATIO (95% CI): Treatment 1 VS. Treatment 2 (Male)","%.2f (%.2f %.2f)"%(c1[0],c1[1],c1[2])])
y.add_row(["ODDS RATIO (95% CI): Treatment 1 VS. Treatment 2 (Female)","%.2f (%.2f %.2f)"%(c2[0],c2[1],c2[2])])
y.add_row(["ODDS RATIO (95% CI): Female VS. Male (Treatment 1)","%.2f (%.2f %.2f)"%(c3[0],c3[1],c3[2])])
y.add_row(["ODDS RATIO (95% CI): Female VS. Male (Treatment 2)","%.2f (%.2f %.2f)"%(c4[0],c4[1],c4[2])])
y.add_row(["INTERACTION P-VALUE","%.4f"%(pval)])

y.align["RESPONSE RATE"] = "l"
y.align[""] = "l"

print(y.get_string(fields=["RESPONSE RATE", ""]))


+------------------------------------------------------------------------------+
|                  Logistic regression for subjects response                   |
+-----------------------------------------------------------+------------------+
| RESPONSE RATE                                             |                  |
+-----------------------------------------------------------+------------------+
| ODDS RATIO (95% CI): Treatment 1 VS. Treatment 2          | 1.67 (1.06 2.64) |
|                                                           |                  |
| ODDS RATIO (95% CI): Treatment 1 VS. Treatment 2 (Male)   | 1.99 (0.80 4.97) |
| ODDS RATIO (95% CI): Treatment 1 VS. Treatment 2 (Female) | 1.36 (0.69 2.70) |
| ODDS RATIO (95% CI): Female VS. Male (Treatment 1)        | 1.31 (0.52 3.28) |
| ODDS RATIO (95% CI): Female VS. Male (Treatment 2)        | 0.90 (0.44 1.81) |
| INTERACTION P-VALUE                                       | 0.4146           |
+---------------------------

In [9]:
file = open('resp.txt', 'w')
file.write(y.get_string(fields=["RESPONSE RATE", ""]))
file.close()

##  How these 4 odds ratio are related to estimated coefficients?
As we can see, OR for the first three conditions is OR>1, which tells us about the increases the odds of subjects having responses for the respective conditions. Only the 4th condition is OR<1, which tells us about the decreases the odds of subjects having responses for that condition.

##  Definition of the pvalue:
The p-value is the probability of obtaining results at least as extreme as the observed results of a statistical hypothesis test, assuming that the null hypothesis is correct. The p-value is used as an alternative to rejection points to provide the smallest level of significance at which the null hypothesis would be rejected. A smaller p-value means that there is stronger evidence in favor of the alternative hypothesis.

##  Based on this p-value – do we need to update model (simplify it)? Why?
Usually, a significance level of 0.05 works well. A significance level of 0.05 indicates a 5% risk of concluding that an effect exists when there is no actual effect. The null hypothesis for an interaction effect is that the response mean for the level of one factor does not depend on the value of the other factor level. If an interaction term is statistically significant, the relationship between a factor and the response differs by the level of the other factor. 

The p-value for the interaction between TRTPN*gender is 0.415, which indicates that the relationship between treatment and gender doesn't depend on gender. Because the interaction effect between treatment and gender isn't statistically significant, you can interpret the main effects without considering the interaction effect.

Therefore, we can update the model by throwing out the interaction and trying some other methods.

